In [1]:
import pandas as pd
train_split = pd.read_csv('https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_train.tsv', delimiter="\\t")
test_split = pd.read_csv('https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_test.tsv', delimiter="\\t")

/tmp/ipykernel_52276/1580820321.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_split = pd.read_csv('https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_train.tsv', delimiter="\\t")
/tmp/ipykernel_52276/1580820321.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_split = pd.read_csv('https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_test.tsv', delimiter="\\t")


In [2]:
train_split

,content,label
0,اللهم ارزقنا فعل الخيرات وترك المنكرات,Positive
1,كلام سليم والصراحه راحه,Positive
2,حتي في التخلف مش فالحين منيح,Negative
3,شو هل الاهبل,Negative
4,اشتقت لمكه و لراءحه مكه و لالصلاه في مك...,Positive
...,...,...
1435,يا الله استجب دعاءي يالله مادكيت بابك يو...,Positive
1436,لا استطيع اللهم ارزقنا العافيه وشكر نعمهك,Positive
1437,الله يحمي البلد من كل مخرب والله يعطيكي...,Positive
1438,اصلا كتير من البنات بدهن يحكن مع الشباب,Negative


In [3]:
sys_msg = "Respond only positive or negative sentiment: "
def create_chat_prompt(sys_msg, input_text):
    return [
        {"role": "system", "content": sys_msg}, 
        {"role": "user", "content": input_text}
    ]

def create_chat_example(content, label):
    return [
        {"role": "system", "content": content, "name": "example_user"},
        {"role": "system", "content": label, "name": "example_assistant"},
    ]

In [4]:
train_split

,content,label
0,اللهم ارزقنا فعل الخيرات وترك المنكرات,Positive
1,كلام سليم والصراحه راحه,Positive
2,حتي في التخلف مش فالحين منيح,Negative
3,شو هل الاهبل,Negative
4,اشتقت لمكه و لراءحه مكه و لالصلاه في مك...,Positive
...,...,...
1435,يا الله استجب دعاءي يالله مادكيت بابك يو...,Positive
1436,لا استطيع اللهم ارزقنا العافيه وشكر نعمهك,Positive
1437,الله يحمي البلد من كل مخرب والله يعطيكي...,Positive
1438,اصلا كتير من البنات بدهن يحكن مع الشباب,Negative


In [5]:
data_path = "../registry/data/ajgt"
!mkdir -p {data_path}

dev_df = train_split
dev_df["sample"] = dev_df.apply(lambda x: create_chat_example(x['content'], x['label']), axis=1)
dev_df[["sample"]].to_json(f'{data_path}/few_shot.jsonl', lines=True, orient="records",force_ascii=False)

test_df = test_split
test_df["input"] = test_df['content'].apply(lambda x: create_chat_prompt(sys_msg, x))
test_df["ideal"] = test_df['label']
test_df[["input", "ideal"]].to_json(f'{data_path}/samples.jsonl', lines=True, orient="records",force_ascii=False)

In [6]:
import os
os.environ["OPENAI_API_KEY"] = "<openai-key>"
os.environ["EVALS_THREADS"]="1"
os.environ["EVALS_THREAD_TIMEOUT"]="100"

In [13]:
cls_task_specs = """
ajgt:
  id: ajgt.test.v1
  metrics: [accuracy]
  description: Evaluate Arabic text classification
# Define the eval
ajgt.test.v1:
  # Specify the class name as a dotted path to the module and class
  class: evals.elsuite.classification:Classification
  args:
    samples_jsonl: ajgt/samples.jsonl
    few_shot_jsonl: ajgt/few_shot.jsonl # path to few shots file
    num_few_shot: 5 # max few shots to use

""".strip()
with open("../registry/evals/ajgt.yaml", "w") as file:
    file.write(cls_task_specs)

In [14]:
!ls ../eval_results

metrec.jsonl


In [15]:
record_path = "../eval_results_gpt_3_5/ajgt-fewshot-5-temp-1.0.jsonl"
# !oaieval gpt-3.5-turbo-0301 ajgt --record_path {record_path} --seed 41
!oaieval gpt-3.5-turbo-0301 ajgt --record_path {record_path} --seed 41 --max_samples 360 --modelspec_extra_options temperature=1.0

[2023-05-30 06:25:08,053] [registry.py:156] Loading registry from /home/magedsaeed/MyProjects/evals/evals/registry/evals
[2023-05-30 06:25:08,159] [registry.py:156] Loading registry from /home/magedsaeed/.evals/evals
[2023-05-30 06:25:09,426] [oaieval.py:213] Run started: 2305300325092BFHBHDY
[2023-05-30 06:25:09,427] [data.py:75] Fetching ajgt/few_shot.jsonl
[2023-05-30 06:25:09,435] [data.py:75] Fetching ajgt/samples.jsonl
[2023-05-30 06:25:09,436] [eval.py:32] Evaluating 360 samples
[2023-05-30 06:25:09,438] [eval.py:152] Running in threaded mode with 1 threads!
 14%|██████                                    | 52/360 [01:18<09:21,  1.82s/it][2023-05-30 06:26:28,286] [_common.py:105] Backing off openai_chat_completion_create_retrying(...) for 0.4s (openai.error.RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-6VIsbC1WgU4rx2bWplxNV7gP on tokens per min. Limit: 90000 / min. Current: 88009 / min. Contact us through our help center at help.openai.com if yo

In [19]:
import pandas as pd
with open(record_path, "r") as f:
    events_df = pd.read_json(f, lines=True)

In [20]:
events_df[events_df['final_report'].notnull()]['final_report']

1001    {'accuracy': 0.025}
Name: final_report, dtype: object

In [21]:
for i, r in pd.json_normalize(events_df[events_df.type == "sampling"].data).iterrows():
    print(f"Prompt: {r.prompt[-1]}")
    print(f"Sampled: {r.sampled}")
    print(f"{i}","--" * 25)

Prompt: {'role': 'user', 'content': 'حسرتي مع ذا ومع ذا فأنا # معهم مع بعدهم في معمعه'}
Sampled: المتدارك
0 --------------------------------------------------
Prompt: {'role': 'user', 'content': 'ولا أخ عن أخ أو عن أب ولد # وما عمرناه للدنيا فمخترب'}
Sampled: المديد
1 --------------------------------------------------
Prompt: {'role': 'user', 'content': 'كافل دين الله سيف له # لا زال للأمة محبوبا'}
Sampled: المديد
2 --------------------------------------------------
Prompt: {'role': 'user', 'content': 'لها شية ما شئت حسنا ومشية # تبذ الجياد السابقات بها عدوا'}
Sampled: المديد
3 --------------------------------------------------
Prompt: {'role': 'user', 'content': 'وهو بدنياه مولع كلف # يقنع من صيدها بمعراض'}
Sampled: المديد
4 --------------------------------------------------
Prompt: {'role': 'user', 'content': 'ما ضرهم لو رحموا متيما # لم يبق من جثمانه سوى رمق'}
Sampled: المديد
5 --------------------------------------------------
Prompt: {'role': 'user', 'content': 'بدا فتقطعت مهج الغ